In [1]:
import scipy.io
import scipy.signal
import numpy as np
import decimal
import pickle
from matplotlib import pyplot as plt
from matplotlib import rcParams

Простая линейная интерполяция

In [2]:
def interpol(P, P_new, sec):
    P = np.ravel(np.sort(P))
    P_new = np.sort(P_new)
    sec_new = np.zeros(len(P_new))
    for i in range(len(P_new)):
        t = np.where(P <= P_new[i])
        k = np.where(P >= P_new[i])
        if ((len(np.ravel(t)) != 0)&(len(np.ravel(k)) != 0)):
            floor = np.max(t)
            ceil = np.min(k)
            if (floor == ceil):
                sec_new[i] = sec[floor]
            else:
                sec_new[i] = sec[floor] + (sec[ceil] - sec[floor])*(P_new[i] - P[floor])/(P[ceil] - P[floor])
        else:
            floor = np.max(np.where(P <= P_new[i]))
            sec_new[i] = sec[floor]
    return P_new, sec_new

Чтение файла с климатом из MCD

In [3]:
data = np.loadtxt('climate.txt')
alt_new = data[:int(len(data)/4), 0] # h [м]
T_new = data[:int(len(data)/4), 1] # T [К]
P_new = data[int(len(data)/4):int(len(data)/2), 1] # P [Па]
WVMR = data[int(len(data)/2):int(3*len(data)/4), 1] # water vapor volume mixing ratio [mol/mol]
OzVMR = data[int(3*len(data)/4):int(len(data)), 1] # ozone volume mixing ratio [mol/mol]

Чтение файла с изначальными сечениями

In [4]:
data1 = scipy.io.loadmat('secmat_o3.mat') # здесь может быть не только о3, но и любой другой газ
data1['SECmat_o3'] = np.delete(data1['SECmat_o3'], (0), axis=0)
alt = data1['alt']*10**3
T = data1['temp']
P = data1['pres']*10**2 # P [Па]
sec = data1['SECmat_o3']

Создаем массив из новых сечений

In [ ]:
sec_new_all = np.zeros(sec.shape[0])
for j in range(sec.shape[1]):
    alt_new, sec_new = interpol(P, P_new, sec[:, j])
    if (j == 0):
        sec_new_all = sec_new
    else:
        sec_new_all = np.column_stack((sec_new_all, sec_new))

Создание нового mat-файла

In [13]:
data_custom = dict.fromkeys(['sec_new', 'alt', 'pres', 'temp', 'wGas', 'VMR'])
data_custom['alt'] = np.flip(alt_new, axis = 0)
data_custom['pres'] = np.flip(P_new, axis = 0)
data_custom['temp'] = np.flip(T_new, axis = 0)
data_custom['wGas'] = data1['wGas']
data_custom['sec_new'] = sec_new_all
data_custom['VMR'] = np.flip(OzVMR, axis = 0) # озон или вода

In [14]:
data_custom

{'VMR': array([  7.77079000e-09,   1.02697000e-08,   1.29220000e-08,
          1.56176000e-08,   2.04302000e-08,   2.55181000e-08,
          2.98094000e-08,   3.29455000e-08,   3.30289000e-08,
          3.33978000e-08,   3.40060000e-08,   3.24739000e-08,
          3.03957000e-08,   2.93026000e-08,   2.82394000e-08,
          2.68184000e-08,   2.53976000e-08,   2.39788000e-08,
          2.29140000e-08,   2.32692000e-08,   1.93387000e-08,
          9.88210000e-09,   4.94196000e-09,   2.01735000e-09,
          1.34870000e-09,   1.26611000e-09,   1.42455000e-09,
          1.76421000e-09,   2.22660000e-09,   2.75845000e-09,
          3.40708000e-09,   4.29922000e-09,   5.16127000e-09,
          5.68396000e-09,   6.05255000e-09]),
 'alt': array([  7.50297000e+02,   5.78351000e+02,   4.46338000e+02,
          3.43340000e+02,   2.62284000e+02,   1.98377000e+02,
          1.48262000e+02,   1.09421000e+02,   7.98830000e+01,
          5.79103000e+01,   4.18373000e+01,   3.02158000e+01,
          

In [15]:
data1

{'SECmat_o3': array([[  8.23470501e-31,   7.55723963e-31,   7.05397110e-31, ...,
           1.82566402e-28,   4.67246000e-28,   2.02794113e-27],
        [  1.48318958e-30,   1.35924330e-30,   1.26731039e-30, ...,
           3.05731935e-28,   6.63470172e-28,   2.55475432e-27],
        [  2.90746911e-30,   2.66223927e-30,   2.48052161e-30, ...,
           5.56733162e-28,   9.79838977e-28,   3.13177516e-27],
        ..., 
        [  6.04849547e-26,   5.60154224e-26,   5.30899130e-26, ...,
           6.04993039e-24,   6.39456663e-24,   6.81652819e-24],
        [  1.20527987e-25,   1.13441840e-25,   1.09283700e-25, ...,
           9.51547247e-24,   1.00613182e-23,   1.07307701e-23],
        [  2.40365726e-25,   2.30948218e-25,   2.26794802e-25, ...,
           1.37572930e-23,   1.45583587e-23,   1.55432615e-23]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file Platform: nt, Created on: Thu Oct 17 08:41:50 2019',
 '__version__': '1.0',
 'alt': array([[ 100.,   95.,   90.,   85.,   

In [16]:
scipy.io.savemat('secmat_o3_climate.mat', data_custom)